In [43]:
import numpy as np
import pandas as pd
import tsfel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [44]:
# Define data paths
data_path = r"C:\Users\HP\Downloads\UCI HAR Dataset/UCI HAR Dataset/"
inertial_signals = ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x', 'body_gyro_y', 'body_gyro_z']

In [45]:
def load_inertial_signals(folder, signals):
    data = []
    for signal in signals:
        file_path = f"{data_path}{folder}/Inertial Signals/{signal}_{folder}.txt"
        raw_data = np.loadtxt(file_path)
        data.append(raw_data)
    return np.stack(data, axis=-1)  # Shape: (samples, time_steps, channels)

In [46]:
# Load raw inertial signals for train and test sets
X_train_raw = load_inertial_signals("train", inertial_signals)
X_test_raw = load_inertial_signals("test", inertial_signals)

In [47]:
# Load activity labels
y_train = np.loadtxt(data_path + "train/y_train.txt") - 1
y_test = np.loadtxt(data_path + "test/y_test.txt") - 1

In [48]:
# Convert labels to integers for SVM
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [49]:
# Load predefined features 
X_train_features = np.loadtxt(data_path + "train/X_train.txt")
X_test_features = np.loadtxt(data_path + "test/X_test.txt")

In [50]:
# Normalize predefined features
scaler_features = StandardScaler()
X_train_features = scaler_features.fit_transform(X_train_features)
X_test_features = scaler_features.transform(X_test_features)

In [51]:
# Load TSFEL feature configuration
cfg = tsfel.get_features_by_domain()

In [52]:
# Set the sampling frequency 
for domain in cfg:
    for feature in cfg[domain]:
        if "fs" in cfg[domain][feature]:  
            cfg[domain][feature]["fs"] = 50  # Set sampling frequency

In [53]:
def extract_features_tsfel(X_data):
    feature_set = []
    for sample in range(X_data.shape[0]):  # Iterate over each sample
        df_sample = pd.DataFrame(X_data[sample], columns=inertial_signals)  # Convert to DataFrame
        features = tsfel.time_series_features_extractor(cfg, df_sample, fs=50, verbose=0)  # Extract features
        feature_set.append(features.values.flatten())  # Flatten to 1D array
    return np.array(feature_set)  # Shape: (samples, features)

In [54]:
# Extract TSFEL features for training and testing
X_train_tsfel = extract_features_tsfel(X_train_raw)
X_test_tsfel = extract_features_tsfel(X_test_raw)

In [55]:
# Normalize TSFEL features
scaler_tsfel = StandardScaler()
X_train_tsfel = scaler_tsfel.fit_transform(X_train_tsfel)
X_test_tsfel = scaler_tsfel.transform(X_test_tsfel)

In [56]:
# Train SVM on TSFEL Features
svm_tsfel = SVC(kernel='rbf')  # Radial Basis Function Kernel
svm_tsfel.fit(X_train_tsfel, y_train)

SVC()

In [57]:
y_pred_tsfel = svm_tsfel.predict(X_test_tsfel)
acc_tsfel = accuracy_score(y_test, y_pred_tsfel)
print(f"SVM Accuracy on TSFEL-Generated Features: {acc_tsfel * 100:.2f}%")

SVM Accuracy on TSFEL-Generated Features: 91.21%


In [58]:
# Train SVM on Predefined Features
svm_predefined = SVC(kernel='rbf')
svm_predefined.fit(X_train_features, y_train)

SVC()

In [59]:
y_pred_predefined = svm_predefined.predict(X_test_features)
acc_predefined = accuracy_score(y_test, y_pred_predefined)
print(f"SVM Accuracy on Predefined Features: {acc_predefined * 100:.2f}%")

SVM Accuracy on Predefined Features: 95.22%


In [60]:
# Compare Results
print("\nComparison of SVM Performance:")
print(f" - SVM with TSFEL Features: {acc_tsfel * 100:.2f}%")
print(f" - SVM with Predefined Features: {acc_predefined * 100:.2f}%")


Comparison of SVM Performance:
 - SVM with TSFEL Features: 91.21%
 - SVM with Predefined Features: 95.22%
